In [75]:
import os
import pandas as pd
import yaml

In [76]:
with open("../../enriched_experiments_new.yml", "r") as f:
    exps = yaml.safe_load(f)

In [77]:
experiments = exps["experiments"]

In [78]:
experiments[-1].keys()

dict_keys(['slurm_id', 'experiment_nm', 'test_nm', 'wandb_urls', 'test/vaec_test5_vit_2/loss', 'val/vaec_vit/loss', 'test/vaec_test3_vit_2/loss', 'test/vaec_test1_vit_2/loss', 'val/loss', 'test/vaec_test4_vit_2/loss', 'train/vaec_vit/loss_epoch', 'test/vaec_test2_vit_2/loss', 'test/vaec_test4_vit_2/accuracy_vaec', 'val/vaec_vit/accuracy_vaec', 'test/vaec_test2_vit_2/accuracy_vaec', 'test/vaec_test1_vit_2/accuracy_vaec', 'test/vaec_test5_vit_2/accuracy_vaec', 'train/vaec_vit/accuracy_vaec_epoch', 'test/vaec_test3_vit_2/accuracy_vaec', 'max_epoch', 'loss/best_ckpt', 'datasets'])

In [79]:
experiments

[{'slurm_id': [832505],
  'experiment_nm': 'bongard_hoi_vasr_images',
  'test_nm': 'lr_check',
  'failed': True},
 {'slurm_id': [832506],
  'experiment_nm': 'bongard_hoi_vasr_images',
  'test_nm': 'lr_check',
  'failed': True},
 {'slurm_id': [832509],
  'experiment_nm': 'bongard_hoi_vasr_images',
  'test_nm': 'lr_check',
  'failed': True},
 {'slurm_id': [832509],
  'experiment_nm': 'bongard_hoi_vasr_images',
  'test_nm': 'lr_check',
  'failed': True},
 {'slurm_id': [832511],
  'experiment_nm': 'bongard_hoi_vasr_images',
  'test_nm': 'lr_check',
  'failed': True},
 {'slurm_id': [835796, 835797, 835798, 839926, 839927, 839928],
  'experiment_nm': 'bongard_hoi_vasr_images',
  'test_nm': 'number_of_slots',
  'wandb_urls': ['https://wandb.ai/avr_universal/AVR_universal/runs/gzkhftm7',
   'https://wandb.ai/avr_universal/AVR_universal/runs/2od077zb',
   'https://wandb.ai/avr_universal/AVR_universal/runs/9t20x5qq',
   'https://wandb.ai/avr_universal/AVR_universal/runs/ppazc5nj',
   'https://wa

In [80]:
list(filter(lambda x: "accuracy" in x, experiments[-1].keys()))

['test/vaec_test4_vit_2/accuracy_vaec',
 'val/vaec_vit/accuracy_vaec',
 'test/vaec_test2_vit_2/accuracy_vaec',
 'test/vaec_test1_vit_2/accuracy_vaec',
 'test/vaec_test5_vit_2/accuracy_vaec',
 'train/vaec_vit/accuracy_vaec_epoch',
 'test/vaec_test3_vit_2/accuracy_vaec']

In [81]:
# Get all runs that produced some accuracy results
unprocessed_count = 0
processed_count = 0
structured_results = []
for experiment in experiments:
    accuracy_cols = [key for key in experiment.keys() if "accuracy" in key]
    if len(accuracy_cols) == 0:
        # TODO: later collect STSN runs
        unprocessed_count += 1
        continue
    structured_results.append(
        {col: experiment[col]["best_loss"] for col in accuracy_cols}
        | {
            "loss/best_ckpt": experiment.get("loss/best_ckpt"),
            "wandb_urls": ",".join(experiment.get("wandb_urls")),
            "experiment_nm": experiment.get("experiment_nm"),
            "test_nm": experiment.get("test_nm"),
            "based_on/slurm_id": experiment.get("based_on", {}).get("slurm_id"),
            "max_epoch": experiment.get("max_epoch"),
        }
    )
    # TODO: enrich with information from "additional_inforamations" (TODO: add relevant fields in enrich_experiments.py)
    processed_count += 1
print(f"Processed {processed_count} experiments")
print(f"Unprocessed {unprocessed_count} experiments")

Processed 348 experiments
Unprocessed 34 experiments


In [82]:
# TODO: split by dataset (not mutually exclusive -- lets duplicate the information)
# VAEC/VASR/HOI/LOGO/LOGO+HOI/VAEC+VASR/LOGO+VAEC/HOI+VASR

# might be nice to add identifiers different than slurm_ids
# (autoincrement value would work but could be hard to
# automatically assign it so that it makes sense in paper)

In [83]:
df = pd.DataFrame(structured_results)

In [84]:
df

,train/bongard_hoi/accuracy_epoch,test/bongard_hoi_seen-seen/accuracy,val/bongard_hoi/accuracy,test/bongard_hoi_seen-unseen/accuracy,test/bongard_hoi_unseen-unseen/accuracy,test/bongard_hoi_unseen-seen/accuracy,loss/best_ckpt,wandb_urls,experiment_nm,test_nm,...,train/svrt_vit_2/accuracy_svrt_epoch,val/svrt_vit_2/accuracy_svrt,test/iraven_vit_2/accuracy_iraven,train/iraven_vit_2/accuracy_iraven_epoch,val/iraven_vit_2/accuracy_iraven,test/bongard_logo_test_bd_vit_2/accuracy_vasr,test/bongard_logo_test_hd_comb_vit_2/accuracy_vaec,test/bongard_logo_test_ff_vit_2/accuracy_vaec,test/bongard_logo_test_bd_vit_2/accuracy_vaec,test/bongard_logo_test_hd_novel_vit_2/accuracy_vaec
0,0.778655,0.489946,0.604056,0.581890,0.502481,0.491890,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring,frozen_slot,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.832082,0.498883,0.580541,0.561531,0.498139,0.474573,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring_full_train,trained_slot_aux_test,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_scoring_full_train,trained_slot_aux_test,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.834469,0.497989,0.607584,0.582802,0.503722,0.513371,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring_full_train,trained_slot_aux_test,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_scoring_full_train,trained_slot_aux_test,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,NaN,NaN,NaN,NaN,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v4,hoi_module_substitute_no_context,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
344,NaN,NaN,NaN,NaN,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_combined_v4,hoi_module_substitute_no_context,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
345,NaN,NaN,NaN,NaN,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v4_agg,vasr_module_substitute_no_context,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346,NaN,NaN,NaN,NaN,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4_agg,vasr_module_substitute_no_context,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# normalize columns

In [85]:
cols = df.columns

In [86]:
cols

Index(['train/bongard_hoi/accuracy_epoch',
       'test/bongard_hoi_seen-seen/accuracy', 'val/bongard_hoi/accuracy',
       'test/bongard_hoi_seen-unseen/accuracy',
       'test/bongard_hoi_unseen-unseen/accuracy',
       'test/bongard_hoi_unseen-seen/accuracy', 'loss/best_ckpt', 'wandb_urls',
       'experiment_nm', 'test_nm',
       ...
       'train/svrt_vit_2/accuracy_svrt_epoch', 'val/svrt_vit_2/accuracy_svrt',
       'test/iraven_vit_2/accuracy_iraven',
       'train/iraven_vit_2/accuracy_iraven_epoch',
       'val/iraven_vit_2/accuracy_iraven',
       'test/bongard_logo_test_bd_vit_2/accuracy_vasr',
       'test/bongard_logo_test_hd_comb_vit_2/accuracy_vaec',
       'test/bongard_logo_test_ff_vit_2/accuracy_vaec',
       'test/bongard_logo_test_bd_vit_2/accuracy_vaec',
       'test/bongard_logo_test_hd_novel_vit_2/accuracy_vaec'],
      dtype='object', length=128)

In [87]:
train_acc = [col for col in cols if "train" in col and "accuracy" in col]
val_acc = [col for col in cols if "val" in col and "accuracy" in col]

In [88]:
datasets = ["bongard_logo", "bongard_hoi", "vaec", "vasr", "iraven", "svrt"]

In [89]:
for dataset in datasets:
    _cols = [col for col in train_acc if dataset in col]
    df[f"train/{dataset}/accuracy"] = df[_cols].bfill(axis=1).iloc[:, 0]

    _cols = [col for col in val_acc if dataset in col]
    df[f"val/{dataset}/accuracy"] = df[_cols].bfill(axis=1).iloc[:, 0]

In [90]:
df = df.drop(
    columns=set(train_acc) - {f"train/{dataset}/accuracy" for dataset in datasets}
)
df = df.drop(columns=set(val_acc) - {f"val/{dataset}/accuracy" for dataset in datasets})

In [91]:
test_acc = [col for col in cols if "test" in col and "accuracy" in col]

In [92]:
test_types = [
    ("bongard_logo", "test_bd"),
    ("bongard_logo", "test_ff"),
    ("bongard_logo", "test_hd_comb"),
    ("bongard_logo", "test_hd_novel"),
    ("vaec", "test1"),
    ("vaec", "test2"),
    ("vaec", "test3"),
    ("vaec", "test4"),
    ("vaec", "test5"),
    ("bongard_hoi", "seen-seen"),
    ("bongard_hoi", "seen-unseen"),
    ("bongard_hoi", "unseen-seen"),
    ("bongard_hoi", "unseen-unseen"),
    ("vasr", "vasr"),
    ("iraven", "iraven"),
    ("svrt", "svrt")
]

In [93]:
for dataset, test_type in test_types:
    _cols = [col for col in test_acc if test_type in col]
    df[f"test/{dataset}/{test_type}"] = df[_cols].bfill(axis=1).iloc[:, 0]

In [94]:
df = df.drop(columns=test_acc)

In [95]:
df.columns

Index(['val/bongard_hoi/accuracy', 'loss/best_ckpt', 'wandb_urls',
       'experiment_nm', 'test_nm', 'based_on/slurm_id', 'max_epoch',
       'val/vasr/accuracy', 'val/bongard_logo/accuracy', 'val/vaec/accuracy',
       'train/bongard_logo/accuracy', 'train/bongard_hoi/accuracy',
       'train/vaec/accuracy', 'train/vasr/accuracy', 'train/iraven/accuracy',
       'val/iraven/accuracy', 'train/svrt/accuracy', 'val/svrt/accuracy',
       'test/bongard_logo/test_bd', 'test/bongard_logo/test_ff',
       'test/bongard_logo/test_hd_comb', 'test/bongard_logo/test_hd_novel',
       'test/vaec/test1', 'test/vaec/test2', 'test/vaec/test3',
       'test/vaec/test4', 'test/vaec/test5', 'test/bongard_hoi/seen-seen',
       'test/bongard_hoi/seen-unseen', 'test/bongard_hoi/unseen-seen',
       'test/bongard_hoi/unseen-unseen', 'test/vasr/vasr',
       'test/iraven/iraven', 'test/svrt/svrt'],
      dtype='object')

In [96]:
df.head()

,val/bongard_hoi/accuracy,loss/best_ckpt,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/vasr/accuracy,val/bongard_logo/accuracy,val/vaec/accuracy,...,test/vaec/test3,test/vaec/test4,test/vaec/test5,test/bongard_hoi/seen-seen,test/bongard_hoi/seen-unseen,test/bongard_hoi/unseen-seen,test/bongard_hoi/unseen-unseen,test/vasr/vasr,test/iraven/iraven,test/svrt/svrt
0,0.604056,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring,frozen_slot,839928.0,43,NaN,NaN,NaN,...,NaN,NaN,NaN,0.489946,0.581890,0.491890,0.502481,NaN,NaN,NaN
1,0.580541,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring_full_train,trained_slot_aux_test,839928.0,42,NaN,NaN,NaN,...,NaN,NaN,NaN,0.498883,0.561531,0.474573,0.498139,NaN,NaN,NaN
2,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_scoring_full_train,trained_slot_aux_test,839928.0,5,0.252280,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.252138,NaN,NaN
3,0.607584,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring_full_train,trained_slot_aux_test,839928.0,34,NaN,NaN,NaN,...,NaN,NaN,NaN,0.497989,0.582802,0.513371,0.503722,NaN,NaN,NaN
4,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_scoring_full_train,trained_slot_aux_test,839928.0,5,0.286448,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.272628,NaN,NaN


In [97]:
for dataset, test_type in test_types:
    _dataset_cols = [col for col in df.columns if dataset in col and "test" not in col]
    cols = [
        "wandb_urls",
        "experiment_nm",
        "test_nm",
        "based_on/slurm_id",
        "max_epoch",
        *_dataset_cols,
    ]
    display(
        df[[f"test/{dataset}/{test_type}"] + cols]
        .sort_values(f"test/{dataset}/{test_type}", ascending=False)
        .head(5)
    )

,test/bongard_logo/test_bd,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/bongard_logo/accuracy,train/bongard_logo/accuracy
25,0.881250,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,frozen_slot,871045.0,413,0.807778,0.822796
54,0.864583,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,trained_slot_aux_test,871045.0,174,0.773333,0.764194
257,0.841667,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v2,asymmetric_only_parameter_search,NaN,116,0.778889,0.976774
269,0.841667,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4,diff_aggregation_parameter_search,NaN,197,0.786667,0.951505
299,0.839583,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4_agg,bongard_hoi_logo_fine_tune,NaN,367,0.784444,0.923763


,test/bongard_logo/test_ff,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/bongard_logo/accuracy,train/bongard_logo/accuracy
25,0.765000,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,frozen_slot,871045.0,413,0.807778,0.822796
41,0.755000,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,trained_slot_aux_test,871039.0,172,0.748889,0.766237
22,0.741667,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,frozen_slot,871037.0,490,0.791111,0.806129
54,0.725000,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,trained_slot_aux_test,871045.0,174,0.773333,0.764194
53,0.721667,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,trained_slot_aux_test,871045.0,255,0.755556,0.764839


,test/bongard_logo/test_hd_comb,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/bongard_logo/accuracy,train/bongard_logo/accuracy
291,0.725,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_transformer_v4,transformer_scoring,NaN,102,0.777778,0.979677
25,0.710,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,frozen_slot,871045.0,413,0.807778,0.822796
273,0.710,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4_agg,no_context_parameter_search,NaN,146,0.784444,0.968817
122,0.705,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_transformer_v2,transformer_scoring_none_activation,NaN,240,0.793333,0.981613
115,0.705,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v2,combined_MLP_scoring_none_activation,NaN,240,0.793333,0.981613


,test/bongard_logo/test_hd_novel,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/bongard_logo/accuracy,train/bongard_logo/accuracy
287,0.765625,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_transformer_v4,small_transformer_scoring,NaN,101,0.763333,0.987097
291,0.765625,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_transformer_v4,transformer_scoring,NaN,102,0.777778,0.979677
257,0.753125,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v2,asymmetric_only_parameter_search,NaN,116,0.778889,0.976774
247,0.743750,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4_agg,baseline,NaN,231,0.797778,0.916344
273,0.743750,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4_agg,no_context_parameter_search,NaN,146,0.784444,0.968817


,test/vaec/test1,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/vaec/accuracy,train/vaec/accuracy
61,0.996722,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_scoring,trained_slot_aux_test,871052.0,114,0.997057,1.0
33,0.994131,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_scoring,train_slot_no_aux,871037.0,73,0.993889,1.0
36,0.988686,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_scoring,train_slot_no_aux,871049.0,92,0.991203,1.0
37,0.988054,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_scoring,train_slot_no_aux,871052.0,115,0.988237,1.0
46,0.986088,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_scoring,trained_slot_aux_test,871039.0,208,0.987817,1.0


...

,test/bongard_hoi/unseen-seen,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/bongard_hoi/accuracy,train/bongard_hoi/accuracy
290,0.761727,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_transformer_v4,transformer_scoring,NaN,54,0.835097,0.997917
279,0.760631,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v4_agg,mlp_hidden_dim_256_128_64,NaN,101,0.837155,0.987891
278,0.754494,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v4_agg,mlp_hidden_dim_256_128,NaN,101,0.827454,0.977909
246,0.751644,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v4_agg,baseline,NaN,101,0.832452,0.979124
188,0.751206,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_vaec_combined_v3,combined_separate_scoring_no_asymetric,NaN,208,0.828924,0.917278


,test/bongard_hoi/unseen-unseen,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/bongard_hoi/accuracy,train/bongard_hoi/accuracy
188,0.819479,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_vaec_combined_v3,combined_separate_scoring_no_asymetric,NaN,208,0.828924,0.917278
131,0.819479,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v2,combined_parameter_search_no_asymetrical,NaN,365,0.830982,0.934899
252,0.818859,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v2,symmetric_only_parameter_search,NaN,119,0.830394,0.920793
186,0.816377,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_bongard_logo_combined_v3,combined_separate_scoring_no_asymetric,NaN,172,0.832745,0.922833
277,0.815757,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v4_agg,mlp_hidden_dim_64,NaN,100,0.820988,0.986589


,test/vasr/vasr,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/vasr/accuracy,train/vasr/accuracy
274,0.595039,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v4,no_context_parameter_search,NaN,100,0.596076,0.956527
248,0.563144,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v4,baseline,NaN,81,0.561884,0.931823
304,0.562642,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v4,vasr_vaec_fine_tune,NaN,61,0.563879,0.913625
283,0.561662,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v4,mlp_hidden_dim_256_128,NaN,75,0.561408,0.977587
335,0.553994,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_vaec_combined_v4,partial_freezing_vaec_0_1,NaN,64,0.569186,0.911837


,test/iraven/iraven,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,train/iraven/accuracy,val/iraven/accuracy
251,0.650822,https://wandb.ai/avr_universal/AVR_universal/r...,iraven_combined_v4,baseline,NaN,499,0.857779,0.650284
0,NaN,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring,frozen_slot,839928.0,43,NaN,NaN
1,NaN,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring_full_train,trained_slot_aux_test,839928.0,42,NaN,NaN
2,NaN,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_scoring_full_train,trained_slot_aux_test,839928.0,5,NaN,NaN
3,NaN,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring_full_train,trained_slot_aux_test,839928.0,34,NaN,NaN


,test/svrt/svrt,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,train/svrt/accuracy,val/svrt/accuracy
250,0.9575,https://wandb.ai/avr_universal/AVR_universal/r...,svrt_combined_v4,baseline,NaN,124,0.9802,0.9705
0,NaN,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring,frozen_slot,839928.0,43,NaN,NaN
1,NaN,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring_full_train,trained_slot_aux_test,839928.0,42,NaN,NaN
2,NaN,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_scoring_full_train,trained_slot_aux_test,839928.0,5,NaN,NaN
3,NaN,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring_full_train,trained_slot_aux_test,839928.0,34,NaN,NaN


In [98]:
df["test/bongard_logo_average"] = 0
df["test/bongard_hoi_average"] = 0
df["test/vaec_average"] = 0

In [99]:
df_zeros = df.fillna(0)

In [100]:
for c in df_zeros.columns:
    if "test/bongard_hoi" in c and c != "test/bongard_hoi_average":
        df_zeros["test/bongard_hoi_average"] += df_zeros[c]

In [101]:
for c in df_zeros.columns:
    if "test/bongard_logo" in c and c != "test/bongard_logo_average":
        df_zeros["test/bongard_logo_average"] += df_zeros[c]

In [109]:
for c in df_zeros.columns:
    if "test/vaec" in c and c != "test/vaec_average":
        df_zeros["test/vaec_average"] += df_zeros[c]

In [ ]:
df_zeros["test/bongard_hoi_average"] /= 4

df_zeros["test/bongard_logo_average"] /= 4

df_zeros["test/vaec_average"] /= 5

In [ ]:
    display(
        df[[f"test/{dataset}/{test_type}"] + cols]
        .sort_values(f"test/{dataset}/{test_type}", ascending=False)
        .head(5)
    )

In [104]:
_dataset_cols = [col for col in df_zeros.columns if "bongard_hoi" in col and "test" not in col]

cols = [
        "wandb_urls",
        "experiment_nm",
        "test_nm",
        "based_on/slurm_id",
        "max_epoch",
        *_dataset_cols,
    ]

display(
    df_zeros[["test/bongard_hoi_average"] + cols]
    .sort_values("test/bongard_hoi_average", ascending=False)
    .head(5)
)

,test/bongard_hoi_average,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/bongard_hoi/accuracy,train/bongard_hoi/accuracy
279,0.803469,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v4_agg,mlp_hidden_dim_256_128_64,0.0,101,0.837155,0.987891
290,0.799484,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_transformer_v4,transformer_scoring,0.0,54,0.835097,0.997917
278,0.799301,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v4_agg,mlp_hidden_dim_256_128,0.0,101,0.827454,0.977909
276,0.794828,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v4_agg,mlp_hidden_dim_256,0.0,100,0.840388,0.982423
318,0.793796,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_bongard_logo_combined_v5,common_relationals_separate_scoring,0.0,101,0.834803,0.988542


In [105]:
_dataset_cols = [col for col in df_zeros.columns if "bongard_logo" in col and "test" not in col]

cols = [
        "wandb_urls",
        "experiment_nm",
        "test_nm",
        "based_on/slurm_id",
        "max_epoch",
        *_dataset_cols,
    ]

display(
    df_zeros[["test/bongard_logo_average"] + cols]
    .sort_values("test/bongard_logo_average", ascending=False)
    .head(5)
)

,test/bongard_logo_average,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/bongard_logo/accuracy,train/bongard_logo/accuracy
25,0.762500,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,frozen_slot,871045.0,413,0.807778,0.822796
291,0.752135,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_transformer_v4,transformer_scoring,0.0,102,0.777778,0.979677
287,0.741927,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_transformer_v4,small_transformer_scoring,0.0,101,0.763333,0.987097
269,0.739167,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4,diff_aggregation_parameter_search,0.0,197,0.786667,0.951505
299,0.736875,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v4_agg,bongard_hoi_logo_fine_tune,0.0,367,0.784444,0.923763


In [112]:
_dataset_cols = [col for col in df_zeros.columns if "vaec" in col and "test" not in col]

cols = [
        "wandb_urls",
        "experiment_nm",
        "test_nm",
        "based_on/slurm_id",
        "max_epoch",
        *_dataset_cols,
    ]

display(
    df_zeros[["test/vaec_average"] + cols]
    .sort_values("test/vaec_average", ascending=False)
    .head(5)
)

,test/vaec_average,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/vaec/accuracy,train/vaec/accuracy
36,0.756908,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_scoring,train_slot_no_aux,871049.0,92,0.991203,1.000000
33,0.665903,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_scoring,train_slot_no_aux,871037.0,73,0.993889,1.000000
255,0.651731,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_combined_v2,symmetric_only_parameter_search,0.0,137,0.639953,0.807199
338,0.648519,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_combined_v4,hoi_module_substitute,0.0,161,0.631460,0.735934
341,0.639195,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_combined_v4,vasr_module_substitute,0.0,179,0.619735,0.736673


In [115]:
a = df_zeros[(df_zeros["experiment_nm"] == "vaec_scoring")&(df_zeros["test_nm"] == "train_slot_no_aux")]

In [117]:
b = df_zeros[(df_zeros["experiment_nm"] == "vaec_combined_v2")&(df_zeros["test_nm"] == "symmetric_only_parameter_search")]

In [116]:
a[["test/vaec/test3", "test/vaec/test1", "test/vaec/test2", "test/vaec/test4", "test/vaec/test5"]]

,test/vaec/test3,test/vaec/test1,test/vaec/test2,test/vaec/test4,test/vaec/test5
33,0.799392,0.994131,0.868881,0.394303,0.272809
36,0.737220,0.988686,0.887547,0.646045,0.525042
37,0.354742,0.988054,0.741521,0.356835,0.582979


In [118]:
b[["test/vaec/test3", "test/vaec/test1", "test/vaec/test2", "test/vaec/test4", "test/vaec/test5"]]

,test/vaec/test3,test/vaec/test1,test/vaec/test2,test/vaec/test4,test/vaec/test5
255,0.67833,0.63839,0.668788,0.611865,0.661281


In [ ]:
for dataset, test_type in test_types:
    _dataset_cols = [col for col in df.columns if dataset in col and "test" not in col]
    cols = [
        "wandb_urls",
        "experiment_nm",
        "test_nm",
        "based_on/slurm_id",
        "max_epoch",
        *_dataset_cols,
    ]
    display(
        df[[f"test/{dataset}/{test_type}"] + cols]
        .sort_values(f"test/{dataset}/{test_type}", ascending=False)
        .head(5)
    )

In [71]:
df[~df["test/vaec/accuracy"].isnull()]

,val/bongard_hoi/accuracy,train/bongard_hoi/accuracy_epoch,loss/best_ckpt,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,train/vasr/accuracy_epoch,val/vasr/accuracy,...,test/vaec_test1_vit_2/accuracy_vasr,test/vaec_test3_vit_2/accuracy_vasr,test/vaec_test4_vit_2/accuracy_vasr,train/bongard_logo/accuracy,train/bongard_hoi/accuracy,train/vaec/accuracy,train/vasr/accuracy,test/bongard_logo_average,test/bongard_hoi_average,test/vaec_average
33,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_scoring,train_slot_no_aux,871037.0,73,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0,0,0
37,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_scoring,train_slot_no_aux,871052.0,115,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0,0,0


In [30]:
df

,val/bongard_hoi/accuracy,train/bongard_hoi/accuracy_epoch,loss/best_ckpt,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,train/vasr/accuracy_epoch,val/vasr/accuracy,...,test/vaec_test1_vit_2/accuracy_vasr,test/vaec_test3_vit_2/accuracy_vasr,test/vaec_test4_vit_2/accuracy_vasr,train/bongard_logo/accuracy,train/bongard_hoi/accuracy,train/vaec/accuracy,train/vasr/accuracy,test/bongard_logo_average,test/bongard_hoi_average,test/vaec_average
0,0.604056,0.778655,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring,frozen_slot,839928.0,43,NaN,NaN,...,NaN,NaN,NaN,NaN,0.778655,NaN,NaN,0,0,0
1,0.580541,0.832082,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring_full_train,trained_slot_aux_test,839928.0,42,NaN,NaN,...,NaN,NaN,NaN,NaN,0.832082,NaN,NaN,0,0,0
2,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_scoring_full_train,trained_slot_aux_test,839928.0,5,0.809366,0.252280,...,NaN,NaN,NaN,NaN,NaN,NaN,0.809366,0,0,0
3,0.607584,0.834469,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring_full_train,trained_slot_aux_test,839928.0,34,NaN,NaN,...,NaN,NaN,NaN,NaN,0.834469,NaN,NaN,0,0,0
4,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_scoring_full_train,trained_slot_aux_test,839928.0,5,0.803173,0.286448,...,NaN,NaN,NaN,NaN,NaN,NaN,0.803173,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v2,combined_vasr_vaec_fine_tune_no_asymetric,NaN,282,NaN,NaN,...,NaN,NaN,NaN,0.783441,NaN,NaN,NaN,0,0,0
204,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v2,combined_vasr_vaec_fine_tune_no_asymetric,NaN,52,NaN,0.453441,...,0.296061,0.312903,0.278003,NaN,NaN,NaN,0.583347,0,0,0
205,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_combined_v2,combined_vasr_vaec_fine_tune_no_asymetric,NaN,214,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.793207,NaN,0,0,0
206,0.738977,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_baseline_v2,None,NaN,33,NaN,NaN,...,NaN,NaN,NaN,NaN,0.995052,NaN,NaN,0,0,0


In [ ]:
df["test/bongard_logo/test_bd"]

InvalidIndexError: ('test/bongard_logo/test_bd', slice(None, None, None))

In [ ]:
#df.to_csv("../../structured_results.csv", index=False)

In [ ]:
#df[["test/bongard_logo/test_bd",	"test/bongard_logo/test_ff",	"test/bongard_logo/test_hd_comb",	"test/bongard_logo/test_hd_novel"]] = df[["test/bongard_logo/test_bd",	"test/bongard_logo/test_ff",	"test/bongard_logo/test_hd_comb",	"test/bongard_logo/test_hd_novel"]].apply(lambda x: round(100*x,1))

In [ ]:
#df[["test/vaec/test1", "test/vaec/test2", "test/vaec/test3", "test/vaec/test4", "test/vaec/test5", "test/bongard_hoi/seen-seen", "test/bongard_hoi/seen-unseen", "test/bongard_hoi/unseen-seen", "test/bongard_hoi/unseen-unseen"]] = df[["test/vaec/test1", "test/vaec/test2", "test/vaec/test3", "test/vaec/test4", "test/vaec/test5", "test/bongard_hoi/seen-seen", "test/bongard_hoi/seen-unseen", "test/bongard_hoi/unseen-seen", "test/bongard_hoi/unseen-unseen"]].apply(lambda x: round(x*100,1))


In [ ]:
# split by dataset 

In [ ]:
#df = round(df * 100,1)